# 4.1 The Metacircular Evaluator

In [ ]:
(define (eval exp env)
  (cond ((self-evaluating? exp)
         exp)
        ((variable? exp)
         (lookup-variable-value exp ev))
        ((quoted? exp)
         (text-of-quotation exp))
        ;; assignment/definition recursively call eval to comput
        ;; new value associated with a variable
        ((assignment? exp)
         (eval-assignment exp env))
        ((definition? exp)
         (eval-definition exp env))
        ((if? exp)
         (eval-if exp env))
        ;; transform into applicable procedure within environment
        ((lambda? exp)
         (make-procedure
          (lambda-parameters exp)
          (lambda-body exp)
          env))
        ;; evaluate sequence of expressions in order
        ((begin? exp)
         (eval-sequence
          (begin-actions exp)
          env))
        ;; case analysis -> nest of if expressions which are then evaluated
        ((cond? exp)
         (eval (cond->if exp) env))
        ((application? exp)
         (apply (eval (operator exp) env)
                (list-of-values
                 (operands exp)
                 env)))
        (else
         (error "Unkown expression
                type: EVAL" exp))))

(define (apply procedure arguments)
  (cond ((primitive-procedure? procedure)
         (apply-primitive-procedure
          procedure
          arguments))
        ;; sequentially eval each expression in procedure body
        ;; extend base environment to include new frame binding
        ;; params of procedure to the arguments it is applied to
        ((compound-procedure? procedure)
         (eval-sequence
          (procedure-body procedure)
          (extend-environment
           (procedure-parameters
            procedure)
           arguments
           (procedure-environment
            procedure))))
        (else
         (error "Unkown procedure
                type: APPLY"
                procedure))))

;; produce list of arguments to which a procedure is to be applied
;; evaluate each operand -> return list of results
(define (list-of-values exps env)
  (if (no-operands? exps)
      '()
      (cons (eval (first-operand exps) env)
            (list-of-values
             (rest-operands exps)
             env))))

(define (eval-if exp env)
  (if (true? (eval (if-predicate exp) env))
      (eval (if-consequent exp) env)
      (eval (if-alternative exp) env)))

;; eval a sequence of expressions in order
;; return value of final expression
(define (eval-sequence exps env)
  (cond ((last-exp? exps)
         (eval (first-exp exps) env))
        (else
         (eval (first-exp exps) env)
         (eval-sequence (rest-exps exps)
                        env))))

;; assign value to variable
(define (eval-assignment exp env)
  (set-variable-value!
   (assignment-variable exp)
   (eval (assignment-value exp) env)
   env)
  'ok)

;; define a variable
(define (eval-definition exp env)
  (define-variable!
   (definition-variable exp)
   (eval (definition-value exp) env)
   env)
  'ok)

## 4.1
Nested `let`s will force the operands to be evaluated in the desired order. The `let` must be nested as multiple assignments in a single let will use the underlying Scheme order of evaluation:
```scheme
(let ((foo exp1) ((bar exp2)))
  ...)
```
Is equivalent to:
```scheme
((lambda (foo bar)...) exp1 exp2)
```

In [1]:
(define (list-of-values-lr exps env)
  (if (no-operands? exps)
      '()
      (let ((left (eval (first-operand exps) env)))
        (let ((right (list-of-values-lr (rest-operands exps) env)))
          (cons left right)))))

(define (list-of-values-rl exps env)
  (if (no-operands? exps)
      '()
      (let ((right (list-of-values-lr (rest-operands exps) env)))
        (let ((left (eval (first-operand exps) env)))
          (cons right left)))))